
Что нужно сделать

Чтобы решить аналитическую задачу, недостаточно сформировать витрину данных из разрозненных файлов. Вам предстоит проанализировать таблицу, которую вы составили в предыдущем блоке, и определить, насколько она пригодна для работы.

Данные, с которыми вы работали в предыдущем модуле, поступили вам напрямую от заказчика. В них могут быть пропуски и ошибки в названиях элементов колонок. Постарайтесь найти и исправить все некорректные строки. Учтите, что просто удалить строки с пропусками недостаточно.
Клиент уточнил, что данные, поступившие от канала «Другое», его не интересуют. Посмотрите, сколько строк с таким значением в вашей таблице. Если вы всё сделали верно, в таблице таких строк не окажется. Почему?
Посчитайте, сколько уникальных пользователей пришло по каждому рекламному каналу в разрезе этапов воронки продаж. Посмотрите на результаты и подумайте, не кажутся ли они вам странными. Не забывайте о последовательности этапов и логике в воронке продаж.
Вы разобрались с ошибками, теперь стоит проанализировать данные на фальсификацию. Клиент считает, что данные некоторых этапов воронки продаж могли быть сфальсифицированы «накликиванием». Это значит, что по рекламным ссылкам переходили специальные боты, а не живые люди. Постарайтесь проверить, так ли это, и удалить строки с фродовыми данными. Для этого проанализируйте, сколько действий в один момент времени совершает пользователь.
Постройте гистограмму числа действий, совершаемых пользователями в один момент времени. Вручную установите количество бинов для достоверного анализа.
Посчитайте процент фродовых пользователей от общего числа. Сохраните фродовые данные в отдельную таблицу, она потребуется в одном из следующих заданий.

In [1]:
import pandas as pd
import numpy as np

In [6]:
my_df = pd.read_excel('result_df.xlsx')
my_df.head(5)

,user_id,funnel_stage,timestamp,profit,os,ad_channel,price
0,666743,interest,150000,NaN,Android,banner,0.5
1,47061,interest,150000,NaN,Android,banner,0.5
2,587720,interest,150000,NaN,Android,banner,0.5
3,991300,intent,150001,NaN,iOS,banner,0.5
4,289020,interest,150001,NaN,Android,banner,0.5


In [7]:
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600889 entries, 0 to 600888
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   user_id       600889 non-null  int64  
 1   funnel_stage  600889 non-null  object 
 2   timestamp     600889 non-null  int64  
 3   profit        1789 non-null    float64
 4   os            600889 non-null  object 
 5   ad_channel    600889 non-null  object 
 6   price         595546 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 32.1+ MB


In [8]:
new_df = my_df[my_df.ad_channel != 'other']
new_df.shape

(595546, 7)

In [9]:
new_df.profit = new_df.profit.fillna(value=0)

C:\Users\semen\AppData\Local\Temp\ipykernel_8396\3518473730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.profit = new_df.profit.fillna(value=0)


In [10]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 595546 entries, 0 to 600888
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   user_id       595546 non-null  int64  
 1   funnel_stage  595546 non-null  object 
 2   timestamp     595546 non-null  int64  
 3   profit        595546 non-null  float64
 4   os            595546 non-null  object 
 5   ad_channel    595546 non-null  object 
 6   price         595546 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 36.3+ MB


In [11]:
new_df.timestamp.value_counts()

timestamp
159868    29
196178    28
171025    27
164492    27
161779    27
          ..
200642     1
200636     1
200613     1
200605     1
200979     1
Name: count, Length: 50911, dtype: int64

In [12]:
new_df['funnel_stage'] = new_df['funnel_stage'].str.lower()

C:\Users\semen\AppData\Local\Temp\ipykernel_8396\1795375983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['funnel_stage'] = new_df['funnel_stage'].str.lower()


In [13]:
new_df.funnel_stage.value_counts()

funnel_stage
interest         366726
consideration    203928
intent            23147
purchase           1745
Name: count, dtype: int64

In [14]:
new_df.groupby(['ad_channel', 'funnel_stage'])['user_id'].nunique()


ad_channel       funnel_stage 
banner           consideration    47454
                 intent            7630
                 interest         92002
                 purchase           690
native           consideration    12585
                 intent             411
                 interest         19382
                 purchase            36
organic_traffic  consideration    53618
                 intent            9164
                 interest         78719
                 purchase           540
partner_network  consideration    24737
                 intent            4422
                 interest         66226
                 purchase           453
rich             consideration     4658
                 intent            1238
                 interest          8921
                 purchase            21
video            consideration    60876
                 intent              58
                 interest         97964
                 purchase             5
Name: use

In [15]:
actions_count = new_df.groupby(['user_id', 'timestamp']).size()

In [16]:
actions_count

user_id  timestamp
4        183254       2
5        153069       1
         153692       1
7        155779       1
8        175997       1
                     ..
1064741  193546       1
         194117       1
1064742  176049       1
1064747  182206       1
         182961       1
Length: 591809, dtype: int64

In [17]:
df_5= actions_count.reset_index(name='count')

In [18]:
df_5

,user_id,timestamp,count
0,4,183254,2
1,5,153069,1
2,5,153692,1
3,7,155779,1
4,8,175997,1
...,...,...,...
591804,1064741,193546,1
591805,1064741,194117,1
591806,1064742,176049,1
591807,1064747,182206,1


In [19]:
df_fraud = df_5[df_5['count'] == 2]

In [22]:
df_fraud

,user_id,timestamp,count
0,4,183254,2
48,73,163846,2
49,73,165066,2
355,549,195410,2
403,645,180672,2
...,...,...,...
591101,1063356,199811,2
591214,1063617,188955,2
591386,1063945,170866,2
591414,1063999,171319,2


In [20]:
result = my_df[my_df['user_id'].isin(df_fraud['user_id'])]

In [21]:
result

,user_id,funnel_stage,timestamp,profit,os,ad_channel,price
114,389186,interest,150050,NaN,Android,banner,0.50
115,389186,interest,150050,NaN,iOS,banner,0.50
284,401967,interest,150139,NaN,iOS,banner,0.50
285,401967,interest,150139,NaN,Android,banner,0.50
293,506048,interest,150145,NaN,iOS,banner,0.50
...,...,...,...,...,...,...,...
599343,472389,interest,199701,NaN,iOS,video,0.75
599611,91301,interest,199788,NaN,Android,video,0.75
599612,91301,interest,199788,NaN,iOS,video,0.75
600146,95565,interest,199962,NaN,Android,video,0.75


In [76]:
print(new_df.shape, result.shape)

(595546, 7) (3737, 3)


In [82]:
print("процент фродовых пользователей =", round(((df_fraud.user_id.nunique())/(new_df.user_id.nunique()) * 100), 3))

процент фродовых пользователей = 0.963


In [23]:
result.to_excel('df_fraud.xlsx')